In [ ]:
#Import required packages
import pandas as pd
import numpy as np
from datetime import datetime
import time

import csv
import os

import mysql.connector
from mysql.connector import Error

from pytz import timezone
from urllib.request import urlopen
import xmltodict
import csv
import time
import requests


#Set min and max journey times possible
min_journey = 180   #Set min journey to 3 minutes
max_journey = 86400 #Max journey 1 day

#set parameters for reading in API and appropriate timezone
URL = 'https://nextbike.net/maps/nextbike-live.xml?city=254'
TIMEOUT = 4
timeZone = 'America/New_York'
setTimeFormat = "%Y-%m-%d %H:%M:%S"

pathName='\\Jupyter\\CREATEJOURNEYSPITTS\\'

In [ ]:
## Database Connection

localhost='pittsburghdb.c9rczggk5uzn.eu-west-2.rds.amazonaws.com'
username='pittsburghadmin'
password='Pitts$8burgh'
databasename='pittsburgh'
port=3306


def create_db_connection(host, user, passwd, database):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host,
            user=user,
            passwd=passwd,
            database=database
        )
        print('Connection to database '+database+' successful')
    except Error as err:
        print(f'An error has been encountered!: "{err}"')
        
    return connection


In [ ]:
def update_bikeslocations_table():
    
    connection = create_db_connection(localhost, username, password, databasename) 
    cursor = connection.cursor()

    cursor.execute('''
                        DELETE FROM bikeslocations
                        ''')

    for row in CurrentBikesLocationsDF.itertuples():
        cursor.execute('''
                        INSERT INTO bikeslocations (bikeid, stationid)
                        VALUES (%s,%s)
                        ''',
                        [row.BikeID, 
                        row.StationID]
                        )
        connection.commit()


In [ ]:
def update_bikesout_table():
    
    connection = create_db_connection(localhost, username, password, databasename) 
    cursor = connection.cursor()
    CurrentBikesOutDF['DateTimeOut']=pd.to_datetime(CurrentBikesOutDF['DateTimeOut']).dt.strftime("%Y-%m-%d %H:%M:%S")

    cursor.execute('''
                        DELETE FROM bikesout
                        ''')

    for row in CurrentBikesOutDF.itertuples():
        cursor.execute('''
                        INSERT INTO bikesout (bikeid, stationid, datetimeout)
                        VALUES (%s,%s,%s)
                        ''',
                        [row.BikeID, 
                        row.StationID,
                        row.DateTimeOut]
                        )
        connection.commit()

In [ ]:
def update_journeys_table():
    
    connection = create_db_connection(localhost, username, password, databasename) 
    cursor = connection.cursor()
    JourneysToAdd['DateTimeOut']=pd.to_datetime(JourneysToAdd['DateTimeOut']).dt.strftime("%Y-%m-%d %H:%M:%S")
    JourneysToAdd['DateTimeIn']=pd.to_datetime(JourneysToAdd['DateTimeIn']).dt.strftime("%Y-%m-%d %H:%M:%S")

    for row in JourneysToAdd.itertuples():
        cursor.execute('''
                        INSERT INTO journeys (bikeid, stationoutid, datetimeout, stationinid, datetimein)
                        VALUES (%s,%s,%s,%s,%s)
                        ''',
                        [row.BikeID, 
                        row.StationOutID,
                        row.DateTimeOut,
                        row.StationInID,
                        row.DateTimeIn]
                        )
        connection.commit()


In [ ]:
def create_backup():
    
    #Bikes API Location
    file = urlopen(URL)
    data = file.read()
    file.close()

    #Parse Data
    data = xmltodict.parse(data)
    newdata=data['markers']['country']['city']['place']

    print(len(newdata))
    #Iterate through all stations in data
    for x in range(len(newdata)):
        initialData=(nowLocal.strftime(setTimeFormat),",",newdata[x].get('@number'),",",newdata[x].get('@name'),",",
                     str(newdata[x].get('@bikes')),",",str(newdata[x].get('@bike_numbers')))
        joinData = "".join(initialData)
        finalData = joinData.split(',')

    #Create or find today's file name 
        nowYear=str(nowLocal.year)
        if now.month >9:
            nowMonth = str(nowLocal.month)
        else:
            nowMonth = "0"+str(nowLocal.month)
        if now.day >9:
            nowDay = str(nowLocal.day)
        else:
            nowDay = "0"+str(nowLocal.day)

        fileName = "BikeData" + nowYear + nowMonth+ nowDay+".csv"  
        print(pathName+fileName)

    #Append to Today's File
        file = open(pathName+fileName, 'a+', newline ='')
        with file:
            write = csv.writer(file)
            write.writerow(finalData)

In [ ]:
CurrentBikesLocationsDF = pd.DataFrame(columns=['BikeID', 'StationID'])           
CurrentBikesOutDF = pd.DataFrame(columns=['BikeID', 'StationID','DateTimeOut'])    
CurrentBikesOutDF['BikeID'] = pd.to_numeric(CurrentBikesOutDF['BikeID'])            
CurrentBikesOutDF['StationID'] = pd.to_numeric(CurrentBikesOutDF['StationID'])      
CurrentBikesOutDF['DateTimeOut'] = pd.to_datetime(CurrentBikesOutDF['DateTimeOut'])    
total_rows=0 #This is just used to monitor script as it runs

#Get stations data into a dataframe
connection = create_db_connection(localhost, username, password, databasename) 
query = "Select * from stations;"
StationsDF =   pd.read_sql(query,connection)


while True:
    
    #Set variable with current local date/time
    now = datetime.now()
    nowLocal = now.astimezone(timezone(timeZone))
    
    try:
    #Make back up to file of latest bike data
        create_backup()
    #################################################

        CurrentDateTime = nowLocal.strftime("%Y-%m-%d %H:%M:%S")      

        PreviousBikesLocationsDF = CurrentBikesLocationsDF        
        PreviousBikesOutDF = CurrentBikesOutDF                    
        CurrentBikesLocationsDF = pd.DataFrame(columns=['BikeID', 'StationID'])    ## Reset Current Bikes Location Dataframe

        #Attempt ways to pick up the element tree
        try:
            import cElementTree as etree
        except ImportError:
            try:
                import xml.etree.cElementTree as etree
            except ImportError:
                exit_err("Failed to import ElementTree")

        #Set the current bike locations     
        req = requests.get(URL, timeout=TIMEOUT)
        data = req.content
        root = etree.fromstring(data)

        for elem in root.iter():
            if "bike_numbers" in elem.attrib.keys():
                numbikes = int(elem.attrib["bikes"])
                individualbikes = str.split(elem.attrib["bike_numbers"],",")
                for i in range (0, numbikes):
                    try:
                        BikeInStation=[individualbikes[i],elem.attrib["number"]]                   
                        if ((StationsDF['stationid'].astype("str").str.contains(elem.attrib["number"])).any() and
                            (individualbikes[i].isspace() == False) and (len(individualbikes[i]) > 1)):
                            CurrentBikesLocationsDF = CurrentBikesLocationsDF.append(pd.DataFrame([BikeInStation]
                                                                              ,columns=['BikeID', 'StationID']),
                                                                              ignore_index = True)
                    except IndexError:
                        print('Index Error')                                                   

        CurrentBikesLocationsDF = CurrentBikesLocationsDF.astype( dtype={'BikeID' :int, 'StationID': int})


        #Find Bikes that are still in a station now and also previous period    
        is_duplicate_in_current = CurrentBikesLocationsDF['BikeID'].isin(PreviousBikesLocationsDF['BikeID'])
        is_duplicate_in_previous = PreviousBikesLocationsDF['BikeID'].isin(CurrentBikesLocationsDF['BikeID'])

        #Bikes that are now in a station, but weren't in previous period are added to returns dataframe
        ReturnsDF = CurrentBikesLocationsDF[~is_duplicate_in_current].copy()
        ReturnsDF = ReturnsDF.reset_index(drop=True) 
        ReturnsDF['DateTimeIn']=pd.to_datetime(CurrentDateTime) 
        ReturnsDF=ReturnsDF.rename(columns={'StationID': 'StationInID','BikeID': 'BikeInID'})


        #Create DF of potential journeys to add to the master journey list
        JourneysToAdd = ReturnsDF.merge(PreviousBikesOutDF, left_on=['BikeInID'], 
                                                  right_on=['BikeID'], how='inner')
        JourneysToAdd.drop('BikeID', axis=1, inplace=True)
        JourneysToAdd=JourneysToAdd.rename(columns={'StationID': 'StationOutID', 'BikeInID':'BikeID'})
        JourneysToAdd['DateTimeOut']=(pd.to_datetime(JourneysToAdd['DateTimeOut'])).dt.strftime("%m/%d/%Y, %H:%M:%S")
        JourneysToAdd['DateTimeOut']=pd.to_datetime(JourneysToAdd['DateTimeOut']) 
        JourneysToAdd['JourneyTime']=(JourneysToAdd['DateTimeIn']-JourneysToAdd['DateTimeOut']).dt.total_seconds().astype(int)
        JourneysToAdd = JourneysToAdd.reindex(columns=['BikeID','StationOutID','DateTimeOut','StationInID','DateTimeIn','JourneyTime'])

        #Updating BikesOut Data by removing bikes that have returned
        CurrentBikesOutDF = PreviousBikesOutDF[PreviousBikesOutDF['BikeID'].isin(JourneysToAdd['BikeID']) == False]

        #Limit trips that are added to Journey table to be in specific range of trip lenghts
        JourneysToAdd= JourneysToAdd[(JourneysToAdd['JourneyTime'] >= min_journey) & (JourneysToAdd['JourneyTime'] <= max_journey)]    
        JourneysToAdd= JourneysToAdd[JourneysToAdd['StationOutID'] != JourneysToAdd['StationInID']]
        JourneysToAdd.drop('JourneyTime', axis=1, inplace=True)   #drop later after bikesout
        JourneysToAdd = JourneysToAdd.reset_index(drop=True) 


        #Bikes to Add to Bikes Out 
        BikesGoingOutDF = PreviousBikesLocationsDF[~is_duplicate_in_previous].copy()

        BikesGoingOutDF['DateTimeOut'] = None
        BikesGoingOutDF['DateTimeOut']=(pd.to_datetime(BikesGoingOutDF['DateTimeOut'])).dt.strftime("%m/%d/%Y, %H:%M:%S")
        BikesGoingOutDF['DateTimeOut']= pd.to_datetime(CurrentDateTime)
        BikesGoingOutDF.reset_index(drop=True, inplace=True)
        print(len(is_duplicate_in_previous), np.sum(is_duplicate_in_previous)) ## monitor script as it runs
        total_rows += len(BikesGoingOutDF.index) #monitor script as it runs
        print(total_rows) #monitor script as it runs

        #Append the new bikes going out to the Current Bikes Out DF
        CurrentBikesOutDF=CurrentBikesOutDF.append(BikesGoingOutDF, ignore_index = True)

  
    ###########################################################################################    
        #Update Tables
        update_bikeslocations_table()
        update_bikesout_table()
        update_journeys_table()
    except:
        print('Error unable to update tables')

    timeStamp=nowLocal.timestamp()
    time.sleep(60.0 - ((time.time() - timeStamp) % 60.0))